## 0. Base Setting

In [1]:
# Google drive 연동

from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Mounted at /gdrive


In [2]:
## Yolov5 git clone 및 requirements install

#%cd /content
#!git clone https://github.com/ultralytics/yolov5.git

%cd /
%cd /gdrive/MyDrive/yolov5/


#설치 후 runtime 초기화
#!pip install -r requirements.txt

/
/gdrive/MyDrive/yolov5


##1. Data preparation

In [3]:
## Google drive의 각 폴더에 raw img, label txt 파일을 준비
## 폴더의 파일명 및 수량은 반드시 동일해야함.

from glob import glob
from sklearn.model_selection import train_test_split

img_list = glob('/gdrive/MyDrive/yolov5_dataset/images/*.jpg')
img_list = sorted(img_list)

print()

txt_list = glob('/gdrive/MyDrive/yolov5_dataset/labels/*.txt')
txt_list = sorted(txt_list)
print(len(img_list), " / ", len(txt_list))

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)
print(len(train_img_list), len(val_img_list))



1142  /  1142
913 229


In [4]:
## image 수량이 안맞을 경우 비교하기위한 func

img_li = list(map(lambda x : x[42:52], img_list))
txt_li = list(map(lambda x : x[42:52], txt_list))

for i in range(len(img_li)):
    if (img_li[i] == txt_li[i]) == False:
        print(i)
    elif i == (len(img_li)-1):
        print('end')

print(img_li[0:5])
print(txt_li[0:5])

end
['0810093029', '0810093103', '0810093137', '0810093202', '0810093236']
['0810093029', '0810093103', '0810093137', '0810093202', '0810093236']


##2. Data input 정보 생성(.yaml)
!!!! 반드시 첨부된 data.yaml 파일 사용

In [5]:
with open('train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')
  
with open('val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [12]:
import yaml

with open('/gdrive/MyDrive/yolov5_dataset/data_custom.yaml', 'r') as f:
  data = yaml.full_load(f)

print(data)

data['train'] = '/gdrive/MyDrive/yolov5/train.txt'
data['val'] = '/gdrive/MyDrive/yolov5/val.txt'

with open('data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'train': '../train/images', 'val': '../valid/images', 'nc': 3, 'names': ['Tire', 'Tire_inner', 'Barcode']}
{'train': '/gdrive/MyDrive/yolov5/train.txt', 'val': '/gdrive/MyDrive/yolov5/val.txt', 'nc': 3, 'names': ['Tire', 'Tire_inner', 'Barcode']}


##3. Training

 자세한 사항은 세부 항복은 train.py의 arg parser 확인
 
 --img : 이미지 처리 size, yolov5 github 참조
 
 --data : yolov5에 custom 된 yaml 파일
 
 --cfg : weights s에 대한 yaml 파일
 
 --weights : weights에 대한 부분은 yolov5 github 참조
 
 --name : train된 best.py가 저장될 folder 이름 ./runs/train/ 에 해당폴더이름으로 저장됨

In [13]:
!python train.py --img 640 --batch 32 --epochs 2 --data data.yaml --cfg ./models/yolov5s.yaml --weights ./weights/yolov5s.pt --name yolov5s_test

github: ⚠️ WARNING: code is out of date by 607 commits. Use 'git pull' to update or 'git clone https://github.com/ultralytics/yolov5' to download latest.
YOLOv5 🚀 2021-5-4 torch 1.10.0+cu111 CPU

Namespace(adam=False, artifact_alias='latest', batch_size=32, bbox_interval=-1, bucket='', cache_images=False, cfg='./models/yolov5s.yaml', data='data.yaml', device='', entity=None, epochs=2, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='yolov5s_test', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/yolov5s_test4', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=32, upload_dataset=False, weights='./weights/yolov5s.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: l

##4. Detecting

 자세한 사항은 세부 항복은 detect.py의 arg parser 확인
 
 --img : 이미지 처리 size, yolov5 github 참조
 
 --conf : bounding box할 treshold 값 (0~1 사이의 값)
 
 --source : detect할 image folder 또는 name
 
 해당폴더이름으로 저장됨

In [14]:


!python detect.py --weights runs/train/yolov5s_10005/weights/best.pt --img 640 --conf 0.25 --source '/gdrive/MyDrive/yolov5_dataset/images_test/'

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, hide_conf=False, hide_labels=False, img_size=640, iou_thres=0.45, line_thickness=3, name='exp', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='/gdrive/MyDrive/yolov5_dataset/images_test/', update=False, view_img=False, weights=['runs/train/yolov5s_10005/weights/best.pt'])
YOLOv5 🚀 2021-5-4 torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 224 layers, 7056607 parameters, 0 gradients
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
image 1/1664 /gdrive/MyDrive/yolov5_dataset/images_test/img_0813120010.jpg: 480x640 1 Barcode, 1 Tire, Done. (0.114s)
image 2/1664 /gdrive/MyDriv

KeyboardInterrupt: ignored